Домашнее задание к семинару 02 (HW02)
Тема: работа с табличными данными в Pandas, контроль качества данных, базовый EDA и визуализация в Matplotlib.

2.3. Содержание ноутбука HW02.ipynb (основная часть)
В ноутбуке homeworks/HW02/HW02.ipynb необходимо выполнить следующие действия.


2.3.1. Загрузка данных и первичный осмотр


In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('S02-hw-dataset.csv', sep=',')
data

,user_id,age,country,purchases,revenue
0,1,25.0,FR,7,749
1,2,24.0,RU,5,1115
2,3,52.0,FR,7,399
3,4,31.0,RU,6,654
4,5,NaN,DE,6,1296
5,6,120.0,FR,-1,785
6,7,46.0,RU,0,0
7,8,28.0,CN,2,456
8,9,39.0,US,4,980
9,10,24.0,RU,7,511


In [6]:
data.head(n = 10)

,user_id,age,country,purchases,revenue
0,1,25.0,FR,7,749
1,2,24.0,RU,5,1115
2,3,52.0,FR,7,399
3,4,31.0,RU,6,654
4,5,NaN,DE,6,1296
5,6,120.0,FR,-1,785
6,7,46.0,RU,0,0
7,8,28.0,CN,2,456
8,9,39.0,US,4,980
9,10,24.0,RU,7,511


In [7]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    41 non-null     int64  
 1   age        39 non-null     float64
 2   country    41 non-null     object 
 3   purchases  41 non-null     int64  
 4   revenue    41 non-null     int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.7+ KB


,user_id,age,purchases,revenue
count,41.000000,39.000000,41.000000,41.000000
mean,20.243902,36.512821,4.829268,820.048780
std,11.659289,18.304259,2.710189,613.127269
min,1.000000,5.000000,-1.000000,0.000000
25%,10.000000,24.000000,3.000000,432.000000
50%,20.000000,33.000000,5.000000,693.000000
75%,30.000000,45.500000,7.000000,1115.000000
max,40.000000,120.000000,11.000000,2475.000000


2.3.2. Пропуски, дубликаты и базовый контроль качества

In [8]:
fract = data.isna().mean()
print('доля пропусков:\n',fract)

доля пропусков:
 user_id      0.00000
age          0.04878
country      0.00000
purchases    0.00000
revenue      0.00000
dtype: float64


In [9]:
print("дубликатов:", data.duplicated().sum())
data[data.duplicated(keep=False)]

дубликатов: 1


,user_id,age,country,purchases,revenue
9,10,24.0,RU,7,511
40,10,24.0,RU,7,511


In [11]:
def detect_issues(data):
    labels = []
    if (data[['purchases', 'age', 'user_id', 'revenue']] < 0).any():
        labels.append('negative_values')
    if data.isna().any():
        labels.append('missing_values')
    if data['age'] > 120:
        labels.append('age>120')
    if (data['revenue'] == 0) and (data['purchases'] > 0):
        labels.append('revenue0_but_purchases>0')
    return ';'.join(labels) if labels else np.nan

issues = data.apply(detect_issues, axis=1)
problematic = data[issues.notna()].copy()
problematic['issues'] = issues[issues.notna()]
problematic

,user_id,age,country,purchases,revenue,issues
4,5,NaN,DE,6,1296,missing_values
5,6,120.0,FR,-1,785,negative_values
10,11,24.0,US,4,0,revenue0_but_purchases>0
12,13,NaN,US,9,630,missing_values


Основные проблемы включают систематические пропуски в ключевых полях - значения возраста отсутствуют в нескольких записях (пользователи 4 и 13), что затрудняет сегментацию и возрастной анализ. Обнаружены логические противоречия и некорректные данные - отрицательное количество покупок у пользователя 6, что технически невозможно, а также случаи нулевой выручки при наличии покупок у пользователя 11, что указывает на ошибки в системе учета или сборе данных. Дополнительную озабоченность вызывают потенциальные аномалии - возраст пользователя 6 составляет 120 лет, что требует проверки на достоверность.